# Using statsmodels SARIMAX class for ARIMA based modelling
Some examples from <a href="https://www.statsmodels.org/dev/examples/notebooks/generated/statespace_sarimax_stata.html">statsmodels.org</a>.

The following explanation of SARIMAX is based on <a href="https://machinelearningmastery.com/sarima-for-time-series-forecasting-in-python/">machinelearningmastery.com</a>.

### Why use the SARIMAX class instead of the ARIMA class?

### SARIMAX
Seasonal autoregressive integrated moving average with exogenous variables (SARIMAX) models are used to model univariate time series' with exogenous variables. SARIMAX can be used to model AR, MA, ARIMA, SARIMA, ARIMAX and SARIMAX models by setting all seasonal parameters to 0.

##### ARIMA based parameters
$p$: Trend autoregression order. 

$d$: Trend difference order.

$q$: Trend moving average order.

##### Seasonal parameters
$P$: Seasonal autoregressive order.

$D$: Seasonal difference order.

$Q$: Seasonal moving average order.

$m$: The number of time steps for a single seasonal period.

# Procedure

1. Test time series for stationarity with Dicky-Fuller test.
2. If data is not stationary, perform various transforms to make it stationary. See other resources.
3. Use plots of the autocorrelation function (ACF) and partial autocorrelation function (PACF) to determine ARIMA parameters. Alternatively, use automated fitting procedures such as a grid search. Note that this risks overfitting.
4. Build and evaluate SARIMA model.

In [ ]:
# Run this cell to install required packages

!pip install numpy
!pip install pandas
!pip install scipy
!pip install matplotlib
!pip install statsmodels

In [2]:
import numpy as np
import pandas as pd
from scipy.stats import norm
import statsmodels.api as sm
import matplotlib.pyplot as plt
from datetime import datetime
import requests
from io import BytesIO

# SARIMA Grid Search
This implementation does not consider exogenous variables.

### Expected Input Data
The expected input is a 1-D time series.

In [5]:
from math import sqrt
from multiprocessing import cpu_count
from joblib import Parallel
from joblib import delayed
from warnings import catch_warnings
from warnings import filterwarnings
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error
 
# one-step sarima forecast
def sarima_forecast(history, config):
    order, sorder, trend = config
    # define model
    model = SARIMAX(history, order=order, seasonal_order=sorder, trend=trend, enforce_stationarity=False, enforce_invertibility=False)
    # fit model
    model_fit = model.fit(disp=False)
    # make one step forecast
    yhat = model_fit.predict(len(history), len(history))
    
    return yhat[0]
 
# root mean squared error or rmse
def measure_rmse(actual, predicted):
    return sqrt(mean_squared_error(actual, predicted))
 
# split a univariate dataset into train/test sets
def train_test_split(data, n_test):
    return data[:-n_test], data[-n_test:]
 
# walk-forward validation for univariate data
def walk_forward_validation(data, n_test, cfg):
    predictions = list()
    # split dataset
    train, test = train_test_split(data, n_test)
    # seed history with training dataset
    history = [x for x in train]
    # step over each time-step in the test set
    for i in range(len(test)):
        # fit model and make forecast for history
        yhat = sarima_forecast(history, cfg)
        # store forecast in list of predictions
        predictions.append(yhat)
        # add actual observation to history for the next loop
        history.append(test[i])
    # estimate prediction error
    error = measure_rmse(test, predictions)
    return error
 
# score a model, return None on failure
def score_model(data, n_test, cfg, debug=False):
    result = None
    # convert config to a key
    key = str(cfg)
    # show all warnings and fail on exception if debugging
    if debug:
        result = walk_forward_validation(data, n_test, cfg)
    else:
        # one failure during model validation suggests an unstable config
        try:
            # never show warnings when grid searching, too noisy
            with catch_warnings():
                filterwarnings("ignore")
                result = walk_forward_validation(data, n_test, cfg)
        except:
            error = None
    # check for an interesting result
    if result is not None:
        print(' > Model[%s] %.3f' % (key, result))
    return (key, result)
 
# grid search configs
def grid_search(data, cfg_list, n_test, parallel=True):
    scores = None
    if parallel:
        # execute configs in parallel
        executor = Parallel(n_jobs=cpu_count(), backend='multiprocessing')
        tasks = (delayed(score_model)(data, n_test, cfg) for cfg in cfg_list)
        scores = executor(tasks)
    else:
        scores = [score_model(data, n_test, cfg) for cfg in cfg_list]
    # remove empty results
    scores = [r for r in scores if r[1] != None]
    # sort configs by error, asc
    scores.sort(key=lambda tup: tup[1])
    return scores
 
# create a set of sarima configs to try
def sarima_configs(seasonal=[0]):
    models = list()
    # define config lists
    p_params = [0, 1, 2]
    d_params = [0, 1]
    q_params = [0, 1, 2]
    t_params = ['n','c','t','ct']
    P_params = [0, 1, 2]
    D_params = [0, 1]
    Q_params = [0, 1, 2]
    m_params = seasonal
    # create config instances
    for p in p_params:
        for d in d_params:
            for q in q_params:
                for t in t_params:
                    for P in P_params:
                        for D in D_params:
                            for Q in Q_params:
                                for m in m_params:
                                    cfg = [(p,d,q), (P,D,Q,m), t]
                                    models.append(cfg)
    return models

# Test if Stationary

In [10]:
from statsmodels.tsa.stattools import adfuller

In [13]:
alpha = 0.05 # significance level
n_test = 4 # Cross validation splits

In [11]:
# define dataset
data = np.sin(np.linspace(0,1000,10000))

In [16]:
augmented_dicky_fuller = adfuller(data)

p_val = augmented_dicky_fuller[1]

if p_val >= alpha:
    print("Dicky-Fuller test failed. Data not stationary")
else:
    print("Dicky-Fuller test passed.")

Dicky-Fuller test passed.


# Grid-Search for Optimal Parameters

In [7]:
# model configs
cfg_list = sarima_configs()
# grid search
scores = grid_search(data, cfg_list, n_test)
print('done')
# list top 3 configs
for cfg, error in scores[:3]:
    print(cfg, error)

[0.         0.09984337 0.19868893 ... 0.69865522 0.76659794 0.82687954]
 > Model[[(0, 0, 0), (0, 0, 0, 0), 'n']] 0.733
 > Model[[(0, 0, 0), (0, 0, 0, 0), 'c']] 0.733
 > Model[[(0, 0, 0), (1, 0, 0, 0), 'n']] 0.075
 > Model[[(0, 0, 0), (2, 0, 0, 0), 'n']] 0.000
 > Model[[(0, 0, 1), (1, 0, 2, 0), 'n']] 0.453
 > Model[[(0, 0, 0), (1, 0, 0, 0), 'c']] 0.075
 > Model[[(0, 0, 0), (1, 0, 1, 0), 'n']] 0.075
 > Model[[(0, 0, 0), (0, 0, 1, 0), 'n']] 0.733
 > Model[[(0, 0, 0), (0, 0, 2, 0), 'n']] 1.265
 > Model[[(0, 0, 0), (0, 0, 0, 0), 'ct']] 0.738
 > Model[[(0, 0, 0), (2, 0, 1, 0), 'n']] 0.000
 > Model[[(0, 0, 1), (0, 0, 0, 0), 't']] 0.735
 > Model[[(0, 0, 0), (1, 0, 2, 0), 'n']] 0.453
 > Model[[(0, 1, 0), (1, 0, 0, 0), 'n']] 0.006
 > Model[[(0, 0, 1), (2, 0, 0, 0), 'n']] 0.000
 > Model[[(0, 0, 1), (2, 0, 0, 0), 'ct']] 0.005
 > Model[[(0, 0, 0), (0, 0, 2, 0), 'c']] 1.265
 > Model[[(0, 0, 0), (0, 0, 1, 0), 'c']] 0.733
 > Model[[(0, 1, 0), (1, 0, 1, 0), 'n']] 0.006
 > Model[[(0, 0, 2), (0, 0, 2, 0)

KeyboardInterrupt: 